**取得google drive認證**

In [0]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


**設定path(本目錄的絕對路徑)**


**下載 SimpleITK套件**

In [0]:
path="drive/My Drive/CT"
!pip install SimpleITK

     |████████████████████████████████| 42.5MB 76kB/s 


**設定3D vgg16**

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten,Dropout
from keras.layers import Conv3D
from keras.layers import MaxPooling3D

input_shape = (128, 128, 64, 1)

model = Sequential([
    Conv3D(64, (2, 2,2), input_shape=input_shape, padding='same',
           activation='relu'),
    Conv3D(64, (2, 2,2), activation='relu', padding='same'),
    MaxPooling3D(pool_size=(2, 2,2), strides=(2, 2,2)),
    Conv3D(128, (2, 2,2), activation='relu', padding='same'),
    Conv3D(128, (2, 2,2), activation='relu', padding='same',),
    MaxPooling3D(pool_size=(2, 2,2), strides=(2, 2,2)),
    Conv3D(256, (2, 2,2), activation='relu', padding='same',),
    Conv3D(256, (2, 2,2), activation='relu', padding='same',),
    Conv3D(256, (2, 2,2), activation='relu', padding='same',),
    MaxPooling3D(pool_size=(2, 2,2), strides=(2, 2,2)),
    Conv3D(512, (2, 2,2), activation='relu', padding='same',),
    Conv3D(512, (2, 2,2), activation='relu', padding='same',),
    Conv3D(512, (2, 2,2), activation='relu', padding='same',),
    MaxPooling3D(pool_size=(2, 2,2), strides=(2, 2,2)),
    Conv3D(512, (2, 2,2), activation='relu', padding='same',),
    Conv3D(512, (2, 2,2), activation='relu', padding='same',),
    Conv3D(512, (2, 2,2), activation='relu', padding='same',),
    MaxPooling3D(pool_size=(2, 2,2), strides=(2, 2,2)),
    Flatten(),
    Dense(4096, activation='relu'),
    Dense(4096, activation='relu'),
    Dropout(0.25),
    Dense(4, activation='softmax')
])

model.summary()

Using TensorFlow backend.






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 128, 128, 64, 64)  576       
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 128, 128, 64, 64)  32832     
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 64, 64, 32, 64)    0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 64, 64, 32, 128)   65664     
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 64, 64, 32, 128)   131200    
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 32, 32, 16, 128)   0         
_

**匯入資料**

In [0]:
import pandas as pd
import SimpleITK as sitk
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import numpy as np

#設定label
label_df=pd.read_csv(path+"label.csv")
labels=label_df['Case']

#onehot encoding 
labels_onehot=np_utils.to_categorical(labels,4)

#宣告一個list存放mask
trainmasks_list=list()

#讀進資料
for m in label_df['mask_name']:
  print("讀進:",m)
  trainmask = sitk.ReadImage(path+"/xVertSeg.v1/Data1/bone_masks/"+str(m)+".mhd")

  trainmasks=sitk.GetArrayFromImage(trainmask)
  trainmasks=trainmasks/trainmasks.max()
  print(trainmasks.min())
  print("trainmasks shape:",trainmasks.shape)
  
  new_trainmasks=np.expand_dims(trainmasks,axis=3)
  print("expand trainmasks shape:",new_trainmasks.shape)

  trainmasks_list.append(new_trainmasks)
  bone_masks_data=np.array(trainmasks_list)
print("bone masks data shape:",bone_masks_data.shape)

#切割 raw data為訓練集和驗證集
x_train,x_test,y_train,y_test=train_test_split(bone_masks_data,labels_onehot,test_size=0.2,random_state=2)

讀進: bone_mask_L1_1
0.0
trainmasks shape: (128, 128, 64)
expand trainmasks shape: (128, 128, 64, 1)
讀進: bone_mask_L1_2
0.0
trainmasks shape: (128, 128, 64)
expand trainmasks shape: (128, 128, 64, 1)
讀進: bone_mask_L1_3
0.0
trainmasks shape: (128, 128, 64)
expand trainmasks shape: (128, 128, 64, 1)
讀進: bone_mask_L1_4
0.0
trainmasks shape: (128, 128, 64)
expand trainmasks shape: (128, 128, 64, 1)
讀進: bone_mask_L1_5
0.0
trainmasks shape: (128, 128, 64)
expand trainmasks shape: (128, 128, 64, 1)
讀進: bone_mask_L1_6
0.0
trainmasks shape: (128, 128, 64)
expand trainmasks shape: (128, 128, 64, 1)
讀進: bone_mask_L1_7
0.0
trainmasks shape: (128, 128, 64)
expand trainmasks shape: (128, 128, 64, 1)
讀進: bone_mask_L1_8
0.0
trainmasks shape: (128, 128, 64)
expand trainmasks shape: (128, 128, 64, 1)
讀進: bone_mask_L1_9
0.0
trainmasks shape: (128, 128, 64)
expand trainmasks shape: (128, 128, 64, 1)
讀進: bone_mask_L1_10
0.0
trainmasks shape: (128, 128, 64)
expand trainmasks shape: (128, 128, 64, 1)
讀進: bone_

**開始訓練**

In [0]:
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *

#參數可自行調整
model.compile(optimizer = Adam(lr=1e-5), loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train,y_train,validation_data=(x_test,y_test),batch_size=4,epochs=40)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 60 samples, validate on 15 samples
Epoch 1/40





60/60 [==============================] - 37s 618ms/step - loss: 1.3860 - acc: 0.3333 - val_loss: 1.3859 - val_acc: 0.2000
Epoch 2/40
60/60 [==============================] - 18s 292ms/step - loss: 1.3842 - acc: 0.3833 - val_loss: 1.3839 - val_acc: 0.2000
Epoch 3/40
60/60 [==============================] - 18s 292ms/step - loss: 1.3774 - acc: 0.3833 - val_loss: 1.3728 - val_acc: 0.2000
Epoch 4/40
60/60 [==============================] - 18s 292ms/step - loss: 1.3355 - acc: 0.3833 - val_loss: 1.3371 - val_acc: 0.2000
Epoch 5/40
60/60 [==============================] - 18s 292ms/step - loss: 1.2991 - acc: 0.3833 - val_loss: 1.3102 - val_acc: 0.2000
Epoch 6/40
60/60 [==============================] - 18s 292ms/step - loss: 1.2895 - acc: 0.3833 - val_loss: 1.3103 - val_acc: 0.2000
Epoch 7/40
60/60 [==============================] - 18